In [ ]:
import nltk; nltk.download('stopwords')
!pip install pyLDAvis


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
     |████████████████████████████████| 1.7MB 12.5MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 15.3MB 248kB/s 
     |████████████████████████████████| 9.9MB 49.7MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-cp37-none-any.whl size=136870 sha256=ad44cb78c1ec6552cfae8cc63594cede64f307d196e6077ab520ff0df42d2b8b
  Stored in directory: /root/.cache/pip/wheels/a0/9c/fc/c6e00689d35c82cf96a8adc70edfe7ba7904374fdac3240ac2
Successfully built pyLDAvis
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.20.3 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 1.2.4 which is incompatible.
ERROR: datas

In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [ ]:
df = pd.read_csv('/content/Apple-Twitter-Sentiment-DFE.csv')
print(df.tweet.unique())
df.head()

['#AAPL:The 10 best Steve Jobs emails ever...http://t.co/82G1kL94tx'
 'RT @JPDesloges: Why AAPL Stock Had a Mini-Flash Crash Today $AAPL #aapl\nhttp://t.co/hGFcjYa0E9'
 'My cat only chews @apple cords. Such an #AppleSnob.' ...
 'RT @MMLXIV: there is no avocado emoji may I ask why @apple'
 '@marcbulandr I could not agree more. Between @Apple @Twitter and @IBMWatson only great things will happen. #AppleandIBM #IBMandTwitter'
 "My iPhone 5's photos are no longer downloading automatically to my laptop when I sync it. @apple support is unhelpful. Any ideas?"]


,tweet
0,#AAPL:The 10 best Steve Jobs emails ever...htt...
1,RT @JPDesloges: Why AAPL Stock Had a Mini-Flas...
2,My cat only chews @apple cords. Such an #Apple...
3,I agree with @jimcramer that the #IndividualIn...
4,Nobody expects the Spanish Inquisition #AAPL


In [ ]:
# Convert to list
data = df.tweet.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['#AAPL:The 10 best Steve Jobs emails ever...http://t.co/82G1kL94tx']


In [ ]:
#tokenize each sentence into a list of words
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['aapl', 'the', 'best', 'steve', 'jobs', 'emails', 'ever', 'http', 'co', 'kl', 'tx']]


In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['aapl', 'the', 'best', 'steve_jobs', 'emails', 'ever', 'http', 'co', 'kl', 'tx']


In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['good', 'email', 'ever', 'http', 'co']]


In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]]


In [ ]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('co', 1), ('email', 1), ('ever', 1), ('good', 1), ('http', 1)]]

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.256*"aapl" + 0.049*"really" + 0.031*"detect" + 0.025*"ever" + '
  '0.018*"find" + 0.017*"fibonacci_technical" + 0.015*"peb" + 0.014*"lot" + '
  '0.014*"create" + 0.014*"edkq"'),
 (1,
  '0.610*"co" + 0.033*"still" + 0.029*"buy" + 0.019*"tedlcb" + '
  '0.018*"stockaction_trade" + 0.018*"stocks_mkt" + 0.012*"device" + '
  '0.009*"lag" + 0.009*"post" + 0.008*"card"'),
 (2,
  '0.136*"make" + 0.091*"be" + 0.052*"profit" + 0.034*"will" + 0.033*"come" + '
  '0.025*"upgrade" + 0.017*"twice" + 0.016*"full" + 0.015*"emojis" + '
  '0.015*"check"'),
 (3,
  '0.130*"battery" + 0.057*"stock" + 0.050*"name" + 0.043*"computer" + '
  '0.041*"price" + 0.030*"game" + 0.028*"twitter" + 0.026*"music" + '
  '0.025*"play" + 0.021*"brand"'),
 (4,
  '0.349*"agree" + 0.071*"future" + 0.039*"could" + 0.033*"great" + '
  '0.026*"plan" + 0.024*"see" + 0.020*"week" + 0.020*"reportedly" + '
  '0.017*"security" + 0.016*"search"'),
 (5,
  '0.139*"update" + 0.110*"year" + 0.061*"think" + 0.052*"watch" + '
  '0.

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.08038730871924

Coherence Score:  0.6130584397754935


In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
13     0.125997  0.385772       1        1  9.252953
1      0.380833 -0.168907       2        1  9.234236
15    -0.010055 -0.005570       3        1  7.460877
4      0.006557 -0.003375       4        1  6.000708
16    -0.009990 -0.008950       5        1  5.856950
19     0.058300  0.022429       6        1  5.517747
11     0.060027  0.033221       7        1  5.407621
3     -0.051387 -0.017317       8        1  5.323568
5     -0.022772 -0.013296       9        1  4.524571
2     -0.027433 -0.014558      10        1  4.491015
10    -0.047666 -0.018840      11        1  4.258714
7     -0.041705 -0.021119      12        1  4.180303
14    -0.050591 -0.022353      13        1  3.895208
8     -0.047609 -0.020126      14        1  3.887615
17    -0.047008 -0.021059      15        1  3.870899
12    -0.057458 -0.022850      16        1  3.494043
9     -0.054059 -0.020526      17        1  3.424149
6     -0.055538 -0.021346      18        1  3.365351
0     -0.055351 -0.020894      19        1  3.362982
18    -0.053092 -0.020336      20        1  3.190491, topic_info=           Term         Freq        Total Category  logprob  loglift
0            co  1133.000000  1133.000000  Default  30.0000  30.0000
57        store   445.000000   445.000000  Default  29.0000  29.0000
8         agree   421.000000   421.000000  Default  28.0000  28.0000
4          http   414.000000   414.000000  Default  27.0000  27.0000
16        apple   297.000000   297.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
1523     person     4.731047     5.581131  Topic20  -4.9095   3.2798
408        junk     4.104010     4.954094  Topic20  -5.0517   3.2567
2039  workforce     3.816825     4.666911  Topic20  -5.1243   3.2439
3572        wze     3.652226     4.517537  Topic20  -5.1683   3.2324
3621         op     3.651128     4.519583  Topic20  -5.1686   3.2316

[636 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
12       19  0.993089         aapl
757      11  0.969027         able
671       4  0.940646       access
47        5  0.948057       accord
3623     11  0.869707  acquisition
...     ...       ...          ...
3572     20  0.885438          wze
670       9  0.993337         year
372       7  0.854686          yet
900      18  0.969368     yosemite
3639      7  0.888050         ypfn

[607 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 2, 16, 5, 17, 20, 12, 4, 6, 3, 11, 8, 15, 9, 18, 13, 10, 7, 1, 19])